In [1]:
%load_ext watermark
%watermark -v -m  -u -n -p pandas,numpy,textwrap,matplotlib,sklearn,gseapy,topicpy -a Filippo_Valle -g -r -b -w

Filippo_Valle 
last updated: Tue Jun 30 2020 

CPython 3.7.6
IPython 7.15.0

pandas 1.0.4
numpy 1.18.5
textwrap unknown
matplotlib 3.2.1
sklearn 0.0
gseapy 0.9.18
topicpy 1.0.0

compiler   : GCC 7.5.0
system     : Linux
release    : 4.19.76-linuxkit
machine    : x86_64
processor  : x86_64
CPU cores  : 2
interpreter: 64bit
Git hash   : 3047bd298aea1fc9c7cafce980412f5e9871bca1
Git repo   : git@github.com:fvalle1/topics.git
Git branch : develop
watermark 2.0.2


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from topicpy.hsbmpy import get_max_available_L, get_file
import tensorflow as tf

In [ ]:
algorithm = 'tm'
directory = "/home/jovyan/work/phd/datasets/paper/gtex10seed/"
L=get_max_available_L(directory, algorithm)
L=0
os.chdir(directory)

In [ ]:
df_Pwt = pd.read_csv("%s/%s_level_%d_word-dist.csv"%(algorithm,algorithm,L), index_col=0)

In [ ]:
fig=plt.figure(figsize=(10,8))
plt.title("")
for topic in df_Pwt.columns[:5]:
    dist = df_Pwt.sort_values(by=[topic], ascending=False)[topic].astype(float)
    dist = dist[dist>1e-50].values
    if len(dist)<2:
        pass
    plt.plot(np.sort(dist[dist>0].T)[::-1], marker='x', label=topic)
plt.ylabel("$P(word | topic)$", fontsize=35)
plt.xlabel("words", fontsize=35)
plt.xscale('log')
plt.yscale('log')
plt.tick_params(which="both",labelsize=35)
#plt.legend()
plt.show()
fig.savefig("p_w_tw_%s.pdf"%algorithm)

In [ ]:
df_Ptd = pd.read_csv("%s/%s_level_%d_topic-dist.csv"%(algorithm,algorithm,L), index_col=1).drop('i_doc',1)

In [ ]:
fig=plt.figure(figsize=(20,10))
plt.ylabel("$P(topic | sample$)", fontsize=35)
plt.xlabel("topics", fontsize=35)
for sample in df_Ptd.index.values[:10]:
    plt.plot(df_Ptd.loc[sample,:].astype(float).values, marker='x')
    #plt.plot(np.sort(df_Ptd.filter(F.col('doc')==sample[0]).drop('doc').toPandas().astype(float).values[0])[::-1], lw=0.1, marker='x')
#plt.plot(np.logspace(0,2),1/np.logspace(0,2))
#plt.xscale('log')
#plt.yscale('log')
plt.ylim(1e-4,1)
plt.xlim(0,len(df_Ptd.columns)-2)
plt.tick_params(labelsize=35)
plt.show()
fig.savefig("p_t_s_%s.pdf"%algorithm)

In [ ]:
df_mt = pd.read_csv("mainTable.csv",sep=",",index_col=0)
df_mt.index = [g[:15] for g in df_mt.index]
df_files=pd.read_csv("files.dat", index_col=0)
#df_mt=df_mt.reindex(index=df_Pwt.index.values.ravel(),
#                    columns=[get_file(sample, df_files).name for sample in df_Ptd.index.values.ravel()])
df_mt.dropna(how="all", axis=0, inplace=True)
#df_mt=df_mt.applymap(lambda tpm: np.log2(tpm+1))
#df_mt=df_mt.where(df_mt<1e6,1e6)
df_mt.head()

In [ ]:
Pgt = tf.convert_to_tensor(df_Pwt.values)
Pts = tf.transpose(tf.convert_to_tensor(df_Ptd.values))
Pgs = tf.matmul(Pgt,Pts)
Ps = tf.convert_to_tensor(df_mt.sum(0).values)
Ps = tf.divide(Ps, tf.reduce_sum(Ps))#normalize
Pg = tf.squeeze(tf.matmul(Pgs, tf.reshape(Ps,[Ps.shape[0],1])))

In [ ]:
print(Pgt.shape,Pts.shape,Pgs.shape,Pg.shape, Ps.shape)

In [ ]:
Pgs_data = Pgs.numpy()
Pg_data = Pg.numpy()
Pgs_data.shape

In [ ]:
df_Pgs = pd.DataFrame(data=Pgs_data, index=df_Pwt.values.T[0], columns=df_Ptd.index.values.ravel())

In [ ]:
A=df_mt.fillna(0).astype(int).sum(1).values

In [ ]:
fig=plt.figure(figsize=(10,8))
plt.plot(np.sort(A)[::-1]/A.sum(), lw=15, ls='-', label='data')
plt.plot(np.sort(Pg_data)[::-1]/np.sum(Pg_data), alpha=0.8, lw=15, ls=':', label='P(w)')
plt.ylabel("$P(word)$", fontsize=35)
plt.xlabel("words", fontsize=35)
plt.xscale('log')
plt.yscale('log')
plt.tick_params(labelsize=35)
plt.tick_params(which="both", width=25, length=10, labelsize=35)
plt.legend(fontsize=35)
plt.show()
fig.savefig("p_w_%s.pdf"%algorithm)

In [ ]:
fig=plt.figure(figsize=(10,8))
plt.plot(np.sort(df_mt.mean(1).values)[::-1]/df_mt.mean(1).sum(), lw=15, ls='-', label='data')
plt.plot(np.sort(df_Ptd.transpose().sum(1).values)[::-1]/df_Ptd.transpose().sum(1).sum(), alpha=0.8, lw=15, ls=':', label='P(t|d)')
plt.ylabel("$P(word$)", fontsize=35)
plt.xlabel("words", fontsize=35)
plt.xscale('log')
plt.yscale('log')
plt.tick_params(labelsize=35)
plt.tick_params(which="both", width=25, length=10, labelsize=35)
plt.legend(fontsize=35)
plt.show()
fig.savefig("ptd_w_%s.pdf"%algorithm)

In [ ]:
df_topics = pd.read_csv("%s/%s_level_%d_topics.csv"%(algorithm,algorithm,L))
f = df_mt.sum(1)/df_mt.sum().sum()
f = f.sort_values(ascending=False)

df_Ptd = pd.read_csv("%s/%s_level_%d_topic-dist.csv"%(algorithm,algorithm,L), index_col=1).drop("i_doc",1)
df_Ptd = df_Ptd.transpose()
f_t = df_Ptd.sum(1)
f_t = f_t / f_t.sum()

correlation = []

for topic in df_topics.columns:
    f_g_value = np.mean(f[[g[:15] for g in df_topics[topic].dropna()]])
    f_t_value = f_t[topic]
    correlation.append((f_g_value,f_t_value))

In [ ]:
fig=plt.figure(figsize=(18,15))
plt.scatter(*np.array(correlation).T, color="gray", s=550, label="topic")

plt.xlabel("<Frequency>, $<f_i>_t$", fontsize=35)
plt.ylabel("$<P(topic)>$", fontsize=35)

plt.title(f"{algorithm} - Correlation: {np.round(np.corrcoef(*np.array(correlation).T)[0,1], 2)}", fontsize=35)

#plt.xscale("log")
#plt.yscale("log")

plt.tick_params(which="both", labelsize=35)

plt.legend(fontsize=35)
plt.show()
fig.savefig("corr_f_pt_%s_%d.pdf"%(algorithm,L))